## Symbolic Representation of Frequency Data with MEI and Verovio

This notebook demonstrates methods for converting frequency data into symbolic musical notation using the Music Encoding Initiative (MEI) format and rendering it visually with Verovio. We'll explore different ways to represent pitch, including microtonal alterations, and various display configurations.

### 1. Loading Example Frequency Data

First, let's load some example frequency tables. We'll use two TSV (Tab-Separated Values) files representing a single DFT frame from different audio files with speech contours and percussive sounds. These tables contain columns with frequency values in Hertz (Hz) and corresponding amplitudes.

In [1]:
import pandas as pd

# Load data from TSV files
# Replace 'tsv/speech1.tsv' and 'tsv/conga3.tsv' with the actual paths to your files if needed.
try:
    df_speech = pd.read_csv('tsv/speech1.tsv', delimiter='\t')
    df_percussion = pd.read_csv('tsv/conga3.tsv', delimiter='\t')

    print("Speech Data Sample:")
    display(df_speech.head())
    print("\nPercussion Data Sample:")
    display(df_percussion.head())

except FileNotFoundError:
    print("ERROR: One or more TSV files not found. Please check the file paths.")

Speech Data Sample:


,Frequency (Hz),Amplitude
0,97.368421,0.000965
1,105.263158,0.005521
2,110.526316,0.002423
3,194.736842,0.001351
4,202.631579,0.003517



Percussion Data Sample:


,Frequency (Hz),Amplitude
0,31.25,0.000569
1,75.00,0.000870
2,127.50,0.000540
3,238.75,0.003074
4,321.25,0.002468


### 2. Visualizing Frequencies with `process_and_visualize`

The `process_and_visualize` function is designed to take a DataFrame containing frequency data and render it as musical notation. It offers several options to customize the representation:

**Function Options Breakdown:**

*   **`data`**: (pd.DataFrame or str)
    *   The input data. Can be a pandas DataFrame directly or a string path to a CSV/TSV file.
    *   The DataFrame *must* contain a column named `'Frequency (Hz)'`.

*   **`resolution`**: (str)
    *   Determines how pitches are quantized and what microtonal accidentals are used.
    *   `'half_tone'`: Standard 12-tone equal temperament. Frequencies are mapped to the nearest half-tone.
    *   `'quarter_tone'`: Allows for quarter-tone accidentals (e.g., sharp-and-a-half, flat-and-a-half). Frequencies are mapped to the nearest quarter-tone.
    *   `'eighth_tone'`: Allows for eighth-tone accidentals (using arrows in combination with standard and quarter-tone accidentals). Frequencies are mapped to the nearest eighth-tone.
    *   `'half_tone_deviation'`: Displays notes in standard half-tone resolution but adds a numerical annotation (as a "fingering" mark) above/below each note indicating the deviation in cents from that half-tone. This provides the most precise pitch information visually.

*   **`clef_config`**: (str)
    *   Configures the clef(s) used for rendering.
    *   `'G'`: Single G-clef (treble clef).
    *   `'F'`: Single F-clef (bass clef).
    *   `'GF'`: Grand staff with G-clef on top and F-clef on the bottom, connected by a brace. Notes are automatically placed on the appropriate staff based on their MIDI value (MIDI > 60 typically on G-staff).

*   **`note_order`**: (str)
    *   Determines the order in which notes are processed and displayed, especially relevant for the `'sequence'` display mode or when multiple frequencies are treated as a single chord.
    *   `'original'`: Notes are processed in the order they appear in the input DataFrame.
    *   `'ascending'`: Notes are sorted by pitch (MIDI value, then cent deviation) in ascending order before rendering.
    *   `'descending'`: Notes are sorted by pitch in descending order.

*   **`display_cent_deviation`**: (bool)
    *   Specifically for the `'half_tone_deviation'` resolution.
    *   `True`: Shows the cent deviation numbers above/below notes.
    *   `False`: Hides the cent deviation numbers, even if `resolution` is `'half_tone_deviation'`.
    *   *Note: For `'chord'` display mode, this is often internally forced to `False` to avoid visual clutter, as cent deviations for individual notes within a dense chord can be hard to read.*

*   **`include_natural_accidentals`**: (bool)
    *   `True`: Explicitly shows natural signs (♮) for notes that are natural, even if they wouldn't traditionally require one (e.g., C natural after C sharp).
    *   `False`: Hides natural signs unless strictly necessary according to standard music notation rules (e.g., to cancel a previous accidental within the same measure/context).

*   **`display_mode`**: (str)
    *   Controls how the list of frequencies is visualized.
    *   `'sequence'`: Each frequency from the input is rendered as a separate note event, typically in its own (invisible) measure. This is useful for visualizing melodic lines or series of distinct sonic events.
    *   `'chord'`: All frequencies from the input are rendered simultaneously as a single chord in one measure. This is useful for analyzing harmonic content or simultaneous sounds.

*   **`display_df`**: (bool)
    *   `True`: Displays a pandas DataFrame showing the original frequencies along with their calculated MIDI values, pitch names, and MEI components. This helps in understanding the conversion process.
    *   `False`: Suppresses the display of this intermediate DataFrame.

*   **`save_mei`**: (bool, default: `False`)
    *   If `True`, saves the generated MEI code to a file.
*   **`mei_save_path`**: (str, default: based on function)
    *   The file path where the MEI code will be saved if `save_mei` is `True`.
*   **`save_svg`**: (bool, default: `False`)
    *   If `True`, saves the rendered SVG graphic to a file.
*   **`svg_save_path`**: (str, default: based on function, e.g., `'output.svg'`)
    *   The file path where the SVG graphic will be saved if `save_svg` is `True`.

Let's see some examples using the percussion dataset (`df_percussion`). We will also save the output of the second example as an SVG file.

In [2]:
from py_scripts.mei import process_and_visualize

# Create a 'mei' directory if it doesn't exist
import os
if not os.path.exists('mei'):
    os.makedirs('mei')
    print("Created 'mei' directory for saved files.")

# Example 1: Sequence display with half-tone deviation (not saving SVG here)
print("--- Example 1: Sequence with Half-Tone Deviation ---")
process_and_visualize(
    df_percussion.copy(),
    resolution='half_tone_deviation',
    clef_config='GF',
    note_order='original',
    display_cent_deviation=True,
    include_natural_accidentals=True,
    display_mode='sequence',
    display_df=True,
    save_mei=True,
    mei_save_path='mei/percussion_sequence.mei', # Example path
    save_svg=False
)

# Example 2: Chord display with eighth-tone resolution, SAVING SVG
print("\n--- Example 2: Chord with Eighth-Tone Resolution (Saving SVG) ---")
process_and_visualize(
    df_percussion.copy(),
    resolution='eighth_tone',
    clef_config='GF',
    note_order='ascending',
    display_cent_deviation=False,
    include_natural_accidentals=True,
    display_mode='chord',
    display_df=True,
    save_mei=True,
    mei_save_path='mei/percussion_chord.mei',
    save_svg=True,
    svg_save_path='mei/percussion_chord_eighth_tone.svg'
)

--- Example 1: Sequence with Half-Tone Deviation ---


,Frequency (Hz),MIDI Value,Cent Deviation,Pitch Name,MEI Step,MEI Octave,MEI Alter
0,31.25,23,+21,B0,B,0,n
1,75.00,38,+37,D2,D,2,n
2,127.50,48,-44,C3,C,3,n
3,238.75,58,+42,A#3,A,3,s
4,321.25,64,-45,E4,E,4,n
5,393.75,67,+8,G4,G,4,n
6,398.75,67,+30,G4,G,4,n
7,518.75,72,-15,C5,C,5,n
8,546.25,73,-26,C#5,C,5,s
9,600.00,74,+37,D5,D,5,n



--- Example 2: Chord with Eighth-Tone Resolution (Saving SVG) ---


,Frequency (Hz),MIDI Value,Cent Deviation,Pitch Name,MEI Step,MEI Octave,MEI Alter
0,31.25,23,+21,B0,B,0,nu
1,75.00,38,+37,D2,D,2,1qs
2,127.50,48,-44,C3,C,3,1qf
3,238.75,58,+42,A#3,A,3,3qs
4,321.25,64,-45,E4,E,4,1qf
5,393.75,67,+8,G4,G,4,n
6,398.75,67,+30,G4,G,4,1qs
7,518.75,72,-15,C5,C,5,nd
8,546.25,73,-26,C#5,C,5,n
9,600.00,74,+37,D5,D,5,1qs


**Explanation of the Examples:**

*   **Example 1 (`display_mode='sequence'`, `resolution='half_tone_deviation'`):**
    *   Each frequency in `df_percussion` will be shown as a separate note.
    *   The notes will be placed on either the G or F staff (`clef_config='GF'`).
    *   The pitch will be the closest standard half-tone, and a number above/below it will show the precise cent deviation from that half-tone.
    *   Natural accidentals will be shown.
    *   The intermediate DataFrame will be displayed.
    *   The generated MEI code will be saved to `mei/percussion_sequence.mei` because `save_mei=True`.
    *   The rendered SVG graphic will *not* be saved for this example (`save_svg=False`).

*   **Example 2 (`display_mode='chord'`, `resolution='eighth_tone'`):**
    *   All frequencies in `df_percussion` will be combined into a single chord.
    *   The notes within the chord will be sorted in ascending pitch order.
    *   Eighth-tone accidentals (like up/down arrows) will be used to represent pitches as accurately as possible within that microtonal system.
    *   `display_cent_deviation` is set to `False` as the eighth-tone resolution directly incorporates microtonal accidentals, and numerical cent deviations are primarily for the `'half_tone_deviation'` mode.
    *   The intermediate DataFrame will also be shown.
    *   The generated MEI code will be saved to `mei/percussion_chord.mei` (`save_mei=True`).
    *   Crucially, the rendered musical notation will also be saved as an SVG (Scalable Vector Graphics) file to `mei/percussion_chord_eighth_tone.svg` because `save_svg=True`. This SVG file can be viewed in web browsers or image editors that support SVG.

**Note on Saving Graphics:**

The `save_svg=True` option directly saves the vector-based SVG output from Verovio. If you need a raster format like PNG, you would typically convert the saved SVG file using an external tool or library (e.g., `cairosvg` in Python, or dedicated graphics software).

### 3. Visualizing Longer Sequences and Saving as SVG

The `process_and_visualize` function (and similarly `process_temporal_chords`) is configured with the Verovio option `"breaks": "none"`. This setting forces Verovio to render all the musical events as a single, continuous horizontal system, preventing any line or page breaks.

This behavior is particularly useful when:
1.  You want to see an entire sequence or series of events without interruption.
2.  You intend to save the output as a graphic (like SVG) that captures the full, unbroken representation.

In the following example, we'll use the `df_speech` DataFrame, which contains a larger number of frequency events compared to `df_percussion`. We will visualize it as a sequence and save the resulting continuous SVG graphic.

In [3]:
# Ensure df_speech is loaded from previous cells
from py_scripts.mei import process_and_visualize

# Create the 'mei' directory if it doesn't exist
import os
if not os.path.exists('mei'):
    os.makedirs('mei')
    print("Created 'mei' directory for saved files (if not already present).")

print("--- Visualizing Speech Data as a Long Sequence (Saving MEI and SVG) ---")
process_and_visualize(
    df_speech.copy(),
    resolution='half_tone_deviation',
    clef_config='GF',
    note_order='original',
    display_cent_deviation=True,
    include_natural_accidentals=True,
    display_mode='sequence',
    display_df=False,
    save_mei=True,
    mei_save_path='mei/speech_sequence_htd.mei', # Using htd for half-tone deviation
    save_svg=True,
    svg_save_path='mei/speech_sequence_htd.svg'
)

--- Visualizing Speech Data as a Long Sequence (Saving MEI and SVG) ---


      
**Observations from the Speech Example:**

*   You should see a long, horizontally scrolling musical notation in the notebook output.
*   The file `mei/speech_sequence_htd.svg` will be created. You'll see the full, unbroken sequence there.
*   The `mei/speech_sequence_htd.mei` file will also be saved, containing the underlying MEI data.

This demonstrates how the `"breaks": "none"` setting combined with SVG saving allows for the export of complete, uninterrupted symbolic visualizations, which can be very useful for detailed analysis or for inclusion in publications where a continuous representation is preferred.

### 4. Visualizing Temporal Sequences of Frequency Groups (e.g., DFT Frames)

The `process_temporal_chords` function is tailored for datasets where frequencies are grouped by time windows, such as successive frames from a Short-Time Fourier Transform (STFT/DFT) analysis. Each time group is represented as a chord, and the duration of that time window influences the spacing between these chords in the musical notation.

**Key Aspects for Temporal Representation:**

*   **Input Data Structure:** Requires columns for frequency, start time, and stop time (e.g., `freq_col='freq_start'`, `time_start_col='time_start'`, `time_stop_col='time_stop'`).
*   **Chord Representation:** All frequencies occurring within the same `time_start` and `time_stop` window are rendered as a single chord in one measure.
*   **Temporal Spacing (`measure_represents_sec`):** This parameter (defaulting to 0.5 seconds in the function, but adjustable) controls the visual spacing. The actual duration of each time window (`time_stop - time_start`) is divided by `measure_represents_sec` to determine how many "visual measures" (including padding measures) that event should occupy. This allows for a proportional representation of time.
*   **Resolution and Clarity:**
    *   Since DFT frames can contain many frequencies, using resolutions like `'quarter_tone'` or `'eighth_tone'` is common to capture pitch details.
    *   The `'half_tone_deviation'` mode (displaying numerical cent deviations) is *not* typically used for the chords themselves in this temporal mode, as it would lead to extreme visual clutter. The underlying pitch calculations are still done, but the `<fing>` elements for deviation are usually suppressed.
*   **Breaks:** Similar to `process_and_visualize`, this function also uses `"breaks": "none"` by default in its Verovio options, ensuring a continuous horizontal rendering of the sequence of chords.

**Function Options (Recap of relevant and new parameters):**

*   **`data`, `freq_col`, `time_start_col`, `time_stop_col`**: Define the input and relevant columns.
*   **`resolution`**: Pitch resolution for notes within each chord (e.g., `'quarter_tone'`, `'eighth_tone'`).
*   **`clef_config`**: Clef setup (`'G'`, `'F'`, `'GF'`).
*   **`note_order_within_chord`**: Order of notes within each displayed chord (`'original'`, `'ascending'`, `'descending'`).
*   **`include_natural_accidentals`**: Whether to show explicit natural signs.
*   **`measure_represents_sec`**: The visual duration (in seconds) that each MEI measure (chord or padding) represents, controlling temporal spacing.
*   **`display_df`**: Show summary DataFrame of grouped data.
*   **`save_mei`**: (bool) Save the generated MEI code.
*   **`mei_save_path`**: (str) Path for saving MEI file.
*   **`save_svg`**: (bool) Save the rendered SVG graphic.
*   **`svg_save_path`**: (str) Path for saving SVG file.

**Considerations for DFT/STFT Data:**

*   **Density:** DFT frames can be very dense with frequencies. The resulting chords can be visually complex.
*   **Accuracy vs. Readability:** Higher microtonal resolutions (`'eighth_tone'`) provide more accuracy but can lead to many accidentals, potentially making the score look "skewed" or cluttered if individual notes are very close. The primary goal here is often to see the temporal evolution of these spectral snapshots.

Let's visualize an example using `conga_multidft.tsv`, which represents several successive DFT frames from a conga sound. We'll adjust `measure_represents_sec` to control the visual spacing.

In [7]:
from py_scripts.mei import process_temporal_chords

# Load the multi-DFT frame data
try:
    df_conga_dft = pd.read_csv('tsv/conga_multidft.tsv', delimiter='\t')
    print("Conga Multi-DFT Data Sample:")
    display(df_conga_dft.head())
except FileNotFoundError:
    print("ERROR: 'tsv/conga_multidft.tsv' not found. Please check the file path.")

# Create the 'mei' directory if it doesn't exist
import os
if not os.path.exists('mei'):
    os.makedirs('mei')
    print("Created 'mei' directory for saved files (if not already present).")

print("\n--- Visualizing Temporal Sequence of Conga DFT Frames (Saving MEI and SVG) ---")
process_temporal_chords(
    df_conga_dft.copy(),
    freq_col='freq_start',
    time_start_col='time_start',
    time_stop_col='time_stop',
    resolution='eighth_tone', # Using eighth_tone for microtonal detail
    clef_config='GF',
    note_order_within_chord='ascending',
    include_natural_accidentals=True,
    measure_represents_sec=0.025, # Each measure represents 25ms of sound.
                                 # Adjust this based on the actual durations in your DFT data
                                 # to get desired visual spacing.
    display_df=True,
    save_mei=True,
    mei_save_path='mei/conga_dft_temporal.mei',
    save_svg=True,
    svg_save_path='mei/conga_dft_temporal.svg'
)

Conga Multi-DFT Data Sample:


,freq_start,freq_stop,time_start,time_stop,amp_min,amp_max
0,71.428571,71.428571,0.03,0.1,0.481900,0.481900
1,128.571429,128.571429,0.03,0.1,0.525280,0.525280
2,157.142857,157.142857,0.03,0.1,0.556550,0.556550
3,200.000000,200.000000,0.03,0.1,0.567891,0.567891
4,242.857143,242.857143,0.03,0.1,0.740521,0.740521



--- Visualizing Temporal Sequence of Conga DFT Frames (Saving MEI and SVG) ---


,Group (Measure),Time Start,Time Stop,Num Notes,MIDI Values
0,1,0.03,0.10,51,"[38, 48, 51, 55, 59, 61, 64, 67, 70, 71, 72, 7..."
1,2,0.10,0.20,26,"[23, 39, 47, 51, 54, 56, 59, 61, 63, 66, 67, 6..."
2,3,0.20,0.40,9,"[23, 38, 59, 61, 63, 67, 72, 73, 74]"
3,4,0.40,0.75,1,[59]


      
**Interpreting the Temporal Chord Output:**

*   You will see a sequence of chords. Each chord represents the frequencies present in one time frame (defined by `time_start` and `time_stop`) from `df_conga_dft`.
*   The horizontal spacing between these chords is determined by the actual duration of each frame and the `measure_represents_sec` parameter. If a frame is 50ms long and `measure_represents_sec` is 25ms (0.025s), that frame's chord will be followed by one padding measure (total 2 visual measures: 1 for chord, 1 for padding, representing 50ms).
*   The files `mei/conga_dft_temporal.mei` and `mei/conga_dft_temporal.svg` will be saved, allowing you to inspect the MEI data and the full visual output.
*   The displayed summary DataFrame (if `display_df=True`) will show information about each time group (which becomes a measure in the MEI).

This method provides a powerful way to visualize how the spectral content (represented as chords) evolves over time. Adjusting `measure_represents_sec` is key to achieving a clear and proportional temporal representation.